# Import OBIS data in HDFS

Let's set the up environment variables (be sure the path are good for your system)

In [1]:
import os

os.environ["SPARK_HOME"] = "/opt/spark"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

The dataset can be collected in the OBIS site: https://obis.org/data/access/

In [ ]:
!/usr/local/hadoop/sbin/start-dfs.sh

In [ ]:
!sudo jps

In [4]:
!/usr/local/hadoop/bin/hdfs dfs -mkdir -p /user/OBIS

In [ ]:
!/usr/local/hadoop/bin/hadoop fs -ls /user/OBIS

In [9]:
import findspark

findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

if spark is None:
    # No active session, create a new one
    spark = SparkSession.builder \
        .appName("collect_OBIS") \
        .getOrCreate()
    print("New Spark session created.")
else:
    # An active session already exists
    print("Spark session is already running.")

In [ ]:
# Path to the local CSV file
csv_file_path = "./obis_20230208.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [ ]:
# Path to HDFS directory
hdfs_output_path = "hdfs://localhost:9000/user/OBIS/data"

# Write the DataFrame to HDFS
df.write.csv(hdfs_output_path, header=True)

Let's display 10 rows of  the table:

In [ ]:
hdfs_directory_path = "hdfs://localhost:9000/user/OBIS/data"
# Read files from the HDFS directory into a DataFrame
try:
    df = spark.read.option("header", "true").csv(hdfs_directory_path)
    df.show(10)  
except Exception as e:
    print("Error reading from HDFS:", str(e))

In [12]:
spark.stop()

In [ ]:
!/usr/local/hadoop/sbin/stop-dfs.sh